In [78]:
import pandas as pd
import duckdb
from chess_pipeline import run_pipeline

In [244]:
db = duckdb.connect(":memory:")
db.sql("SET TimeZone='UTC'")

In [245]:
username = 'johnnywhoopp'
# username = 'magnuscarlsen'
run_pipeline(db, username=username)

Getting archive from https://api.chess.com/pub/player/johnnywhoopp/games/2024/04Getting archive from https://api.chess.com/pub/player/johnnywhoopp/games/2024/05

Getting archive from https://api.chess.com/pub/player/johnnywhoopp/games/2024/06
Getting archive from https://api.chess.com/pub/player/johnnywhoopp/games/2024/07
Getting archive from https://api.chess.com/pub/player/johnnywhoopp/games/2024/08
Pipeline chess_pipeline load step completed in 0.23 seconds
1 load package(s) were loaded to destination duckdb and into dataset chess_data
The duckdb destination used duckdb:///:external: location to store data
Load package 1722938796.2975738 is LOADED and contains no failed jobs


In [246]:
db.sql("DESCRIBE;")

┌──────────┬────────────┬─────────────────────┬──────────────────────┬─────────────────────────────────────┬───────────┐
│ database │   schema   │        name         │     column_names     │            column_types             │ temporary │
│ varchar  │  varchar   │       varchar       │      varchar[]       │              varchar[]              │  boolean  │
├──────────┼────────────┼─────────────────────┼──────────────────────┼─────────────────────────────────────┼───────────┤
│ memory   │ chess_data │ _dlt_loads          │ [load_id, schema_n…  │ [VARCHAR, VARCHAR, BIGINT, TIMEST…  │ false     │
│ memory   │ chess_data │ _dlt_pipeline_state │ [version, engine_v…  │ [BIGINT, BIGINT, VARCHAR, VARCHAR…  │ false     │
│ memory   │ chess_data │ _dlt_version        │ [version, engine_v…  │ [BIGINT, BIGINT, TIMESTAMP WITH T…  │ false     │
│ memory   │ chess_data │ player_games        │ [end_time, url, pg…  │ [TIMESTAMP WITH TIME ZONE, VARCHA…  │ false     │
│ memory   │ chess_data │ player

In [247]:
df_player = db.sql("select * from chess_data.player_profile;").to_df()

In [248]:
df_player

,last_online,joined,player_id,aid,url,username,followers,country,status,is_streamer,verified,_dlt_load_id,_dlt_id
0,2024-07-25 14:16:13+00:00,2024-04-03 07:27:43+00:00,354759323,https://api.chess.com/pub/player/johnnywhoopp,https://www.chess.com/member/johnnywhoopp,johnnywhoopp,0,https://api.chess.com/pub/country/JP,basic,False,False,1722938796.2975738,/V90qb9oxRJHpw


In [249]:
df_games = db.sql("select * from chess_data.player_games;").to_df()

In [250]:
df_games.columns

Index(['end_time', 'url', 'pgn', 'time_control', 'rated', 'accuracies__white',
       'accuracies__black', 'tcn', 'uuid', 'initial_setup', 'fen',
       'time_class', 'rules', 'white__rating', 'white__result', 'white__aid',
       'white__username', 'white__uuid', 'black__rating', 'black__result',
       'black__aid', 'black__username', 'black__uuid', '_dlt_load_id',
       '_dlt_id'],
      dtype='object')

In [251]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402 entries, 0 to 401
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   end_time           402 non-null    datetime64[us, UTC]
 1   url                402 non-null    object             
 2   pgn                402 non-null    object             
 3   time_control       402 non-null    object             
 4   rated              402 non-null    bool               
 5   accuracies__white  29 non-null     float64            
 6   accuracies__black  29 non-null     float64            
 7   tcn                402 non-null    object             
 8   uuid               402 non-null    object             
 9   initial_setup      402 non-null    object             
 10  fen                402 non-null    object             
 11  time_class         402 non-null    object             
 12  rules              402 non-null    object         

In [255]:
df_prep = db.sql(f"""
    select
        if(lower(white__username)='{username}', 'White', 'Black') as player_color
        , if(player_color = 'White', white__result, black__result) as player_result
        , if(player_color = 'White', black__result, white__result) as opponent_result
        , case player_result
            -- WIN
            when 'win' then 'win'
            
            -- DRAW
            when 'stalemate' then 'draw'
            when 'agreed' then 'draw'
            when 'repetition' then 'draw'
            when '50move' then 'draw'
            when 'insufficient' then 'draw'
            when 'timevsinsufficient' then 'draw'
            
            -- LOSE
            when 'checkmated' then 'lose'
            when 'timeout' then 'lose'
            when 'resigned' then 'lose'
            when 'abandoned' then 'lose'
            when 'threecheck' then 'lose'
        end as win_draw_lose
        , if(player_result='win', opponent_result, player_result) as reason
        , regexp_extract(pgn, '(ECO )"(.*)"', 2) as eco
        , replace(regexp_extract(pgn, '(UTCDate )"(.*)"', 2), '.', '-') as start_date
        , regexp_extract(pgn, '(StartTime )"(.*)"', 2) as start_time
        , cast(concat(start_date, ' ', start_time) as timestamp) as start_timestamp
        , cast(end_time as timestamp) as end_timestamp
        , age(end_timestamp, start_timestamp) as time_played
        , *
        from chess_data.player_games
""").to_df()

In [257]:
str(df_prep['time_played'].sum())

'3 days 03:14:47'

In [243]:
len(df_prep)

402